In [12]:
import pandas as pd
 
df = pd.read_csv('/Users/mustakahmad/Library/CloudStorage/OneDrive-purdue.edu/FACAI LAB/Project3_tariff/tariff_war/raw_data/soybean/Brazil_Forest_Loss_2013_2022_1km.csv')

In [13]:
df

,system:index,gain,loss,loss_year,treecover2000,.geo
0,0,0,0,2019,100,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,1,0,1,2019,100,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,2,0,0,2019,100,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,3,0,0,2019,100,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,4,0,0,2019,100,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
...,...,...,...,...,...,...
2062992,2062992,0,0,2022,6,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2062993,2062993,0,0,2020,2,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2062994,2062994,0,0,2017,1,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2062995,2062995,0,0,2018,4,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [10]:
import ee
import pandas as pd

# Initialize Earth Engine
ee.Initialize(project= 'forest-cover-461900')

# Load Brazil boundary
brazil = ee.FeatureCollection("FAO/GAUL/2015/level0") \
           .filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))

# Load Hansen dataset
hansen = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')
lossyear = hansen.select('lossyear')
treecover = hansen.select('treecover2000')

# Forest mask and loss year image (masked to forest)
forest_mask = treecover.gt(0)
loss_masked = lossyear.updateMask(forest_mask)

# Pixel area in hectares
pixel_area_ha = ee.Image.pixelArea().divide(10000)
loss_area_ha = pixel_area_ha.updateMask(loss_masked)

# Add actual year band (2000 + lossyear)
year_band = loss_masked.add(2000).rename('Year')
loss_ha_band = loss_area_ha.rename('LossHa')

# Combine bands
loss_image = year_band.addBands(loss_ha_band)

# Reproject to 1km for manageable size
loss_image = loss_image.reproject(crs='EPSG:4326', scale=1000)

# Sample the image
samples = loss_image.sample(
    region=brazil.geometry(),
    scale=1000,
    geometries=True,
    numPixels=1e6,  # cap at 1M pixels to avoid memory issue
    seed=42
)

# Filter to 2015–2022
samples = samples.filter(ee.Filter.gte('Year', 2015)) \
                 .filter(ee.Filter.lte('Year', 2022))

# Get results to client (only works for small exports)
def ee_to_df(feature_coll):
    """Convert FeatureCollection to Pandas DataFrame"""
    features = feature_coll.getInfo()['features']
    data = []
    for f in features:
        props = f['properties']
        coords = f['geometry']['coordinates']
        props['Longitude'] = coords[0]
        props['Latitude'] = coords[1]
        data.append(props)
    return pd.DataFrame(data)

# WARNING: May timeout if too many points; sample small region for testing
df = ee_to_df(samples)

# Save to CSV
df.to_csv('brazil_forest_loss_ha_2015_2022.csv', index=False)

print(df.head())

/Users/mustakahmad/Library/CloudStorage/OneDrive-purdue.edu/FACAI LAB/Project3_tariff/proj3env/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for UMD/hansen/global_forest_change_2022_v1_10! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2022_v1_10

  warnings.warn(warning, category=DeprecationWarning)


EEException: Collection query aborted after accumulating over 5000 elements.

In [11]:
import ee
ee.Initialize(project= 'forest-cover-461900')

# 1. Define Brazil boundary
brazil = ee.FeatureCollection("FAO/GAUL/2015/level0") \
           .filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))

# 2. Load Hansen forest change dataset
hansen = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')

# 3. Select relevant bands
treecover2000 = hansen.select('treecover2000')
loss = hansen.select('loss')
lossyear = hansen.select('lossyear')
gain = hansen.select('gain')

# 4. Mask forest loss from 2015 to 2022
loss_mask = lossyear.gte(15).And(lossyear.lte(22))
loss_filtered = lossyear.updateMask(loss_mask)

# 5. Convert loss year to actual year (2000 + value)
loss_year_actual = loss_filtered.add(2000).rename('loss_year')

# 6. Stack all required bands
stack = treecover2000.addBands(loss).addBands(gain).addBands(loss_year_actual)

# 7. Clip to Brazil and reproject
stack_clipped = stack.clip(brazil).reproject(crs='EPSG:4326', scale=1000)

# 8. Sample pixels
samples = stack_clipped.sample(
    region=brazil.geometry(),
    scale=1000,
    geometries=True
)

# 9. Export to Google Drive as CSV
task = ee.batch.Export.table.toDrive(
    collection=samples,
    description='Brazil_Forest_Loss_2015_2022',
    fileFormat='CSV'
)

task.start()
print("Export started. Check the GEE Code Editor 'Tasks' tab or Google Drive.")

Export started. Check the GEE Code Editor 'Tasks' tab or Google Drive.
